<img src="https://github.com/christopherhuntley/BUAN6510/blob/master/img/Dolan.png?raw=true" width="180px" align="right">

# **BUAN 6510**
# **Lesson 2: Basic SELECT Statements** 
_Retrieving data from a single table._

## **Learning Objectives**
### **Theory / Be able to explain ...**
- The sequence and purpose of each clause of SQL select queries (SELECT, FROM, WHERE, GROUP BY, etc.)
- How SELECT queries compare to similar operations in Excel or Pandas 

### **Skills / Know how to ...**
- Write basic SQL SELECT / FROM / WHERE queries
- Apply functions and conditionals where required
- Calculate aggregate quantities like AVG, SUM, etc.
- Group records using column selectors

--------
## **LESSON 2 HIGHLIGHTS**

In [ ]:
#@title Run this cell if video is does not appear TODO REPLACE WITH NEW VIDEO
%%html
<div style="max-width:1000px">
  <div style="position: relative;padding-bottom: 56.25%;height: 0;">
    <iframe style="position: absolute;top: 0;left: 0;width: 100%;height: 100%;" rel="0" modestbranding="1"  src="https://www.youtube.com/embed/YkDLv6CtEnc" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
  </div>
</div>

## **BIG PICTURE: SQL as a universal data access language** 

In a 






## **SQL History, Standards, and Use Cases**


## **SQL SELECT Statements ... one clause at a time**

### **SELECT**

### **FROM**

### **WHERE**

### **GROUP BY**

### **HAVING**

### **ORDER BY**

### **LIMIT**



## **Aliases and Views**

### **Column Aliases**

### **Expression Aliases**

### **Views (Query Aliases)**

## **Boolean Expressions and WHERE Clauses**

## **Complex Calculations with SQL Functions and CASE Expressions**

## **Grouping and Aggregation**

## **PRO TIPS: How to write queries that work the first time, every time.**

## **SQL AND BEYOND: MySQL DBMS**

## **Congratulations! You've made it to the end of Lesson 2.**

In Lesson 3 we will continue on to more advanced SELECT statements that involve multiple tables or subqueries. That's about as far as most data scientists go ... but we will of course continue on with even more advanced topics. There is *a lot more* to SQL than just retrieving data. 